<a href="https://colab.research.google.com/github/EdnilsonRosa/Data-Science/blob/master/Avalia%C3%A7%C3%A3o%20Big%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avaliação de Big Data

## Inicializando o ambiente e carregando as bibliotecas

In [0]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.2/spark-2.4.2-bin-hadoop2.7.tgz
!tar xf spark-2.4.2-bin-hadoop2.7.tgz
!pip install -q findspark

!java -version

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.11.0-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/content/spark-2.4.2-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

### Chamada da biblioteca

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

### Inicializaçaõ da Sessão do Spark - 

spark = SparkSession.builder \
                    .appName("Tutorial PySpark SQL") \
                    .config("spark.some.config.option", "some-value") \
                    .getOrCreate()

openjdk version "11.0.3" 2019-04-16
OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.04.1)
OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.04.1, mixed mode, sharing)


In [0]:
!pip install jellyfish
import jellyfish

     |████████████████████████████████| 133kB 5.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/eb/33/cf/c7ce9866a02202b1f8ca45595e20f5145bf56c0262cfa2daf1
Successfully built jellyfish


In [0]:
import pandas as pd

## Montando o Drive e carregando o arquivo

In [0]:
### Montagem de drive do Google
### Será solicitada autorização
### O Colab tem como raiz uma pasta 'content'

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
dfa = spark.read.csv('gdrive/My Drive/Colab Notebooks/base_sintetica_ascii_a.csv', header=True, sep=',')

In [0]:
dfb = spark.read.csv('gdrive/My Drive/Colab Notebooks/base_sintetica_ascii_b.csv', header=True, sep=',')

In [0]:
dfb.count()

143

In [0]:
dfa.count()

540

In [0]:
dfb.toPandas()

In [0]:
dfa.show()

+---+--------------------+---------+-----------------+--------------+----------+--------------------+--------------------+------+-----+--------------------+--------------------+------+-----------+--------------+----+
| id|              bairro|      cep|           cidade|           cpf| data_nasc|               email|            endereco|estado|idade|                 mae|                nome|numero|      signo|tipo_sanguineo|_c15|
+---+--------------------+---------+-----------------+--------------+----------+--------------------+--------------------+------+-----+--------------------+--------------------+------+-----------+--------------+----+
|  0|         Vila Sarney|65090-677|         Sao Luis|097.627.958-49|10/09/1948|vicenteolivermont...|2a Travessa do Ri...|    MA|   71|     Louise Beatriz |Vicente Oliver Mo...|   738|     Virgem|           AB-|null|
|  1|      Vila Sao Jorge|75044-220|         Anapolis|685.096.335-09|24/04/1995|miguelbernardogon...|  Praca Joao Salomao|    GO|   

In [0]:
dfa.printSchema()

root
 |-- id: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- email: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- signo: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- _c15: string (nullable = true)



In [0]:
dfb.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)



## Tratamento de Atributo Missing


In [0]:
dfa.filter(F.col('nome').isNull()).count()

0

In [0]:
dfb.filter(F.col('nome').isNull()).count()

2

In [0]:
dfb = dfb.withColumnRenamed('nome','nome_antigo')
dfb = dfb.withColumn('nome', F.when(F.col('nome_antigo').isNull()," ").otherwise(F.col('nome_antigo')))

In [0]:
dfb.show()

+---+---------+-----------------+--------------+----------+--------------------+------+--------------------+--------------------+--------------+--------------------+
| id|      cep|           cidade|           cpf| data_nasc|            endereco|estado|                 mae|         nome_antigo|tipo_sanguineo|                nome|
+---+---------+-----------------+--------------+----------+--------------------+------+--------------------+--------------------+--------------+--------------------+
| 01|69304-350|        Boa Vista|506.865.202-46|22/11/1965|Avenida Mario Hom...|    RR|Isabelle Marcia A...|Felipe Bruno dos ...|           AB-|Felipe Bruno dos ...|
| 02|78132-360|    Varzea Grande|325.828.867-42|      null|      Rua Rio Cuiaba|    MT|       Alana Isadora|Marcio Heitor Sil...|           AB-|Marcio Heitor Sil...|
| 03|88810-336|         Criciuma|284.536.218-88|23/09/1961|Rua Romeu Lopes d...|    SC|  Caroline Gabriela |Theo Mario Victor...|            A-|Theo Mario Victor...|
| 04

## Renomeando colunas para o linkage (necessita nomes diferentes)

In [0]:
dfb = dfb.withColumnRenamed('id','id_b')
dfb = dfb.withColumnRenamed('cep','cep_b')
dfb = dfb.withColumnRenamed('cidade','cidade_b')
dfb = dfb.withColumnRenamed('cpf','cpf_b')
dfb = dfb.withColumnRenamed('data_nasc','data_nasc_b')
dfb = dfb.withColumnRenamed('endereco','endereco_b')
dfb = dfb.withColumnRenamed('estado','estado_b')
dfb = dfb.withColumnRenamed('mae','mae_b')
dfb = dfb.withColumnRenamed('nome','nome_b')
dfb = dfb.withColumnRenamed('tipo_sanguineo','tipo_sanguineo_b')

In [0]:
dfb.printSchema()

root
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_antigo: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- nome_b: string (nullable = true)



## Fazendo Linkage

In [0]:
dfl = dfa.crossJoin(dfb)

In [0]:
dfl.limit(5).show()

+---+-----------+---------+--------+--------------+----------+--------------------+--------------------+------+-----+---------------+--------------------+------+------+--------------+----+----+---------+-------------+--------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------------+
| id|     bairro|      cep|  cidade|           cpf| data_nasc|               email|            endereco|estado|idade|            mae|                nome|numero| signo|tipo_sanguineo|_c15|id_b|    cep_b|     cidade_b|         cpf_b|data_nasc_b|          endereco_b|estado_b|               mae_b|         nome_antigo|tipo_sanguineo_b|              nome_b|
+---+-----------+---------+--------+--------------+----------+--------------------+--------------------+------+-----+---------------+--------------------+------+------+--------------+----+----+---------+-------------+--------------+-----------+--------------------+--------+----------------

In [0]:
dfl.limit(5).toPandas()

In [0]:
dfb.printSchema()
dfa.printSchema()

root
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_antigo: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- nome_b: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- email: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- signo: string (nullable = true)
 |-- tipo_sanguineo: string (

## Criação de Funções para extração do primeiro e último nome

In [0]:
def criaPrimeiroNome(col):
  return col.split(' ') [0]
udf_criaPrimeiroNome = F.udf(criaPrimeiroNome, StringType())

In [0]:
def criaUltimoNome(col):
  return col.split(' ') [-1]
udf_criaUltimoNome = F.udf(criaUltimoNome, StringType())

## Criação das Colunas com Primeiros e Ultimos Nomes Fonéticos

In [0]:
dfl = dfl.withColumn('primeiro_nome_a', udf_criaPrimeiroNome(F.col('nome')))
dfl = dfl.withColumn('primeiro_nome_b', udf_criaPrimeiroNome(F.col('nome_b')))
dfl = dfl.withColumn('ultimo_nome_a', udf_criaUltimoNome(F.col('nome')))
dfl = dfl.withColumn('ultimo_nome_b', udf_criaUltimoNome(F.col('nome_b')))

In [0]:
dfl.show()

+---+-----------+---------+--------+--------------+----------+--------------------+--------------------+------+-----+---------------+--------------------+------+------+--------------+----+----+---------+-----------------+--------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------------+---------------+---------------+-------------+-------------+
| id|     bairro|      cep|  cidade|           cpf| data_nasc|               email|            endereco|estado|idade|            mae|                nome|numero| signo|tipo_sanguineo|_c15|id_b|    cep_b|         cidade_b|         cpf_b|data_nasc_b|          endereco_b|estado_b|               mae_b|         nome_antigo|tipo_sanguineo_b|              nome_b|primeiro_nome_a|primeiro_nome_b|ultimo_nome_a|ultimo_nome_b|
+---+-----------+---------+--------+--------------+----------+--------------------+--------------------+------+-----+---------------+--------------------+------+-

## Criação das colunas com nomes fonéticos

In [0]:
def criaMetaphone(col):
  return jellyfish.metaphone(col)
udf_criaMetaphone = F.udf(criaMetaphone, StringType())

In [0]:
dfl = dfl.withColumn('primeiro_nome_fonetico_a', udf_criaMetaphone(F.col('primeiro_nome_a')))
dfl = dfl.withColumn('primeiro_nome_fonetico_b', udf_criaMetaphone(F.col('primeiro_nome_b')))
dfl = dfl.withColumn('ultimo_nome_fonetico_a', udf_criaMetaphone(F.col('ultimo_nome_a')))
dfl = dfl.withColumn('ultimo_nome_fonetico_b', udf_criaMetaphone(F.col('ultimo_nome_b')))

In [0]:
dfl = dfl.withColumn('nome_fonetico_a', udf_criaMetaphone(F.col('nome')))
dfl = dfl.withColumn('nome_fonetico_b', udf_criaMetaphone(F.col('nome_b')))
dfl = dfl.withColumn('nome_fonetico_mae_a', udf_criaMetaphone(F.col('mae')))
dfl = dfl.withColumn('nome_fonetico_mae_b', udf_criaMetaphone(F.col('mae_b')))

In [0]:
dfl.show()

+---+-----------+---------+--------+--------------+----------+--------------------+--------------------+------+-----+---------------+--------------------+------+------+--------------+----+----+---------+-----------------+--------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------------+---------------+---------------+-------------+-------------+------------------------+------------------------+----------------------+----------------------+---------------+--------------------+-------------------+-------------------+
| id|     bairro|      cep|  cidade|           cpf| data_nasc|               email|            endereco|estado|idade|            mae|                nome|numero| signo|tipo_sanguineo|_c15|id_b|    cep_b|         cidade_b|         cpf_b|data_nasc_b|          endereco_b|estado_b|               mae_b|         nome_antigo|tipo_sanguineo_b|              nome_b|primeiro_nome_a|primeiro_nome_b|ultimo_nome_a|ulti

In [0]:
dfl.printSchema()

root
 |-- id: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- email: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- signo: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_antigo: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable

## Criação da Função de Similaridade

In [0]:
def compare(primeiro_nome_fonetico_a, ultimo_nome_fonetico_a, nome_fonetico_mae_a, primeiro_nome_fonetico_b, ultimo_nome_fonetico_b, nome_fonetico_mae_b):
  sim = 0
  
  #comparando atributos nominais (Phyton 3)
  sim = jellyfish.jaro_winkler(primeiro_nome_fonetico_a.encode('utf-8'), primeiro_nome_fonetico_b.encode('utf-8'))
  sim += jellyfish.jaro_winkler(ultimo_nome_fonetico_a.encode('utf-8'), ultimo_nome_fonetico_b.encode('utf-8'))
  sim += jellyfish.jaro_winkler(nome_fonetico_mae_a.encode('utf-8'), nome_fonetico_mae_b.encode('utf-8'))
  
  #comparando atributos nominais (Phyton 2)
  #  sim = jellyfish.jaro_winkler(primeiro_nome_fonetico_a.encode('utf-8'), primeiro_nome_fonetico_b.encode('utf-8'))
  #  sim += jellyfish.jaro_winkler(unicode(ultimo_nome_fonetico_a), unicode(ultimo_nome_fonetico_b))
  #  sim += jellyfish.jaro_winkler(unicode(nome_fonetico_mae_a), unicode(nome_fonetico_mae_b))
  
  #Media aritmética simples
  sim = str(abs(float(sim/3)))
  
  return sim

udf_compare = F.udf(compare, StringType())                           

In [0]:
# import unicodedata

## Rodando Comparação


In [0]:
dfr = dfl.withColumn('similaridade', udf_compare(F.col('primeiro_nome_fonetico_a'), 
                                                 F.col('ultimo_nome_fonetico_a'), 
                                                 F.col('nome_fonetico_mae_a'), 
                                                 F.col('primeiro_nome_fonetico_b'),
                                                 F.col('ultimo_nome_fonetico_b'),
                                                 F.col('nome_fonetico_mae_b')))

In [0]:
dfr.show()

Py4JJavaError: ignored

In [0]:
dfr.select(['nome','nome_b','cpf','cpf_b', 'similaridade']).show()

Py4JJavaError: ignored

In [0]:
dfr.show()